In [ ]:
import pandas as pd

fake_ds = pd.read_csv('dataset\Fake.csv')
true_ds = pd.read_csv('dataset\True.csv')

fake_ds['label'] = 0
true_ds['label'] = 1

ds = pd.concat([fake_ds, true_ds])
ds = ds.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
import re, nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def process_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return ' '.join([word for word in text.split() if word not in stop_words])
ds['text'] = ds['title']  + ' ' + ds['text']
ds['text'] = ds['text'].apply(process_text)

In [ ]:
from sklearn.model_selection import train_test_split

x = ds['text']
y = ds['label']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42, stratify=y)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7, max_features=5000)
x_train_vec = vectorizer.fit_transform(x_train)
x_test_vec = vectorizer.transform(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(x_train_vec, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_pred = model.predict(x_test_vec)

print('Accuracy:', accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
import seaborn as sb
import matplotlib.pyplot as plot

sb.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues', xticklabels=['Fake', 'True'], yticklabels=['Fake', 'True'])

plot.xlabel('Predicted')
plot.ylabel('Actual')
plot.title('confusion Matrix')
plot.show()

In [ ]:
clean_input = process_text("""
Iran claims its president was injured in Israeli airstrike last month
Iranian President Masoud Pezeshkian suffered a minor leg injury while escaping an Israeli airstrike last month, according to Iran’s semi-official Fars news agency.

The alleged attack happened on June 16 at a secure government facility in Western Tehran, where Pezeshkian was holding a meeting with other senior officials, according to the outlet, which is managed by Iran’s powerful Islamic Revolutionary Guard Corps.

Six bombs or missiles reportedly targeted entry and exit points to block attempts to evacuate the building and disrupt air flow inside. Power was also cut to the targeted floor, after the blasts, the report said.

The officials escaped through an emergency hatch, according to the report. Pezeshkian and several others reportedly suffered minor leg injuries during the escape.
The report claimed the attack was similar to an Israeli plan that led to the assassination of Hezbollah leader Seyed Hassan Nasrallah in September. 
The alleged strike was said to have happened on the fourth day of the 12-day war between Israel and Iran. 

Israel has said its military targeted some of Iran’s top military leaders and nuclear scientists in a series of air strikes aimed at preventing the regime from realizing its goal of producing nuclear weapons.                                                      
""")
input_vector = vectorizer.transform([clean_input])
prediction = model.predict(input_vector)
print("prediction", "Fake" if prediction[0] == 0 else "Real")